In [1]:
import os
import cv2
import numpy as np

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
img_path = 'image_data/train'
os.listdir(path=img_path)

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [4]:
#整理訓練集資料
img_cls = [] 
img_data = []

In [5]:
for idx, dr in enumerate(os.listdir(path=img_path)):
    for filename in os.listdir(path=img_path+'/'+dr):
        img_cls.append(idx)
        
        img = cv2.imread(img_path+'/'+dr+'/'+filename)
        img = cv2.resize(img, (100, 100))
        img_data.append(img)


In [6]:
def list_to_img(img_data):
    dataset = img_data[0][np.newaxis, :]
    for idx, img in enumerate(img_data):
        if idx == 0:
            continue
        else:
            dataset = np.append(dataset, img[np.newaxis, :], axis=0)
    return dataset

In [7]:
x = list_to_img(img_data)

In [8]:
x.shape

(2823, 100, 100, 3)

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, img_cls, test_size=0.2, random_state=22)

In [10]:
from keras.utils import to_categorical
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.
x_test = x_test / 255.

y_train = to_categorical(y_train, 5)
y_test = to_categorical(y_test, 5)

In [11]:
input_shape = x.shape[1:]
input_shape

(100, 100, 3)

In [12]:
# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 9 # 使用 ResNet-56 的網路架構
version = 1
depth = n*6 + 2
model_type = 'ResNet%dv%d' % (depth, version)

In [13]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

In [14]:
batch_size = 32
epochs = 200
data_augmentation = True
num_classes = 5

In [15]:
# 學習率動態調整
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [16]:
# 使用 resnet_layer 來建立我們的 ResNet 模型
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):

    # 建立卷積層
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    # 對輸入進行卷機，根據 conv_first 來決定 conv. bn, activation 的順序
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [17]:
# Resnet v1 共有三個 stage，每經過一次 stage，影像就會變小一半，但 channels 數量增加一倍。ResNet-20 代表共有 20 層 layers，疊越深參數越多
def resnet_v1(input_shape, depth, num_classes=5):

    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # 模型的初始設置，要用多少 filters，共有幾個 residual block （組成 ResNet 的單元）
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    
    # 建立 Input layer
    inputs = Input(shape=input_shape)
    
    # 先對影像做第一次卷機
    x = resnet_layer(inputs=inputs)
    
    # 總共建立 3 個 stage
    for stack in range(3):
        # 每個 stage 建立數個 residual blocks (數量視你的層數而訂，越多層越多 block)
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y]) # 此處把 featuremaps 與 上一層的輸入加起來 (欲更了解結構需閱讀原論文)
            x = Activation('relu')(x)
        num_filters *= 2

    # 建立分類
    # 使用 average pooling，且 size 跟 featuremaps 的 size 一樣 （相等於做 GlobalAveragePooling）
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    
    # 接上 Dense layer 來做分類
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # 建立模型
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [18]:
model = resnet_v1(input_shape=input_shape, depth=depth)

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

Learning rate:  0.001
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 100, 100, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 100, 100, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 100, 100, 16) 0           batch_normalization[0][0]        
_________________________________________________________________

In [20]:
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
callbacks = [lr_reducer, lr_scheduler]


In [21]:
print('Using real-time data augmentation.')
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# 將資料送進 ImageDataGenrator 中做增強
datagen.fit(x_train)

Using real-time data augmentation.


In [22]:
# 訓練模型
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)//batch_size),
                    validation_data=(x_test, y_test),
                    epochs=epochs, verbose=1, workers=4,
                    callbacks=callbacks)

# 評估我們的模型
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Instructions for updating:
Please use Model.fit, which supports generators.
Learning rate:  0.001
Epoch 1/200
70/70 [==============================] - 283s 4s/step - loss: 2.2713 - accuracy: 0.3738 - val_loss: 4.5307 - val_accuracy: 0.2018
Learning rate:  0.001
Epoch 2/200
70/70 [==============================] - 284s 4s/step - loss: 1.7093 - accuracy: 0.5126 - val_loss: 2.2805 - val_accuracy: 0.3168
Learning rate:  0.001
Epoch 3/200
70/70 [==============================] - 282s 4s/step - loss: 1.5811 - accuracy: 0.5557 - val_loss: 2.2121 - val_accuracy: 0.4071
Learning rate:  0.001
Epoch 4/200
70/70 [==============================] - 283s 4s/step - loss: 1.4656 - accuracy: 0.5988 - val_loss: 1.6437 - val_accuracy: 0.5133
Learning rate:  0.001
Epoch 5/200
70/70 [==============================] - 282s 4s/step - loss: 1.3871 - accuracy: 0.6397 - val_loss: 2.7816 - val_accuracy: 0.4513
Learning rate:  0.001
Epoch 6/200
70/70 [==============================] - 282s 4s/step - loss: 1.3208 -

70/70 [==============================] - 306s 4s/step - loss: 0.6889 - accuracy: 0.8625 - val_loss: 1.2990 - val_accuracy: 0.7097
Learning rate:  0.001
Epoch 50/200
70/70 [==============================] - 304s 4s/step - loss: 0.6797 - accuracy: 0.8594 - val_loss: 2.2207 - val_accuracy: 0.6071
Learning rate:  0.001
Epoch 51/200
70/70 [==============================] - 302s 4s/step - loss: 0.7401 - accuracy: 0.8504 - val_loss: 1.2498 - val_accuracy: 0.7522
Learning rate:  0.001
Epoch 52/200
70/70 [==============================] - 304s 4s/step - loss: 0.7017 - accuracy: 0.8634 - val_loss: 1.5268 - val_accuracy: 0.6478
Learning rate:  0.001
Epoch 53/200
70/70 [==============================] - 297s 4s/step - loss: 0.6775 - accuracy: 0.8594 - val_loss: 1.3523 - val_accuracy: 0.7027
Learning rate:  0.001
Epoch 54/200
70/70 [==============================] - 302s 4s/step - loss: 0.6721 - accuracy: 0.8670 - val_loss: 1.3216 - val_accuracy: 0.7150
Learning rate:  0.001
Epoch 55/200
70/70 [===

Learning rate:  0.0001
Epoch 99/200
70/70 [==============================] - 295s 4s/step - loss: 0.3179 - accuracy: 0.9861 - val_loss: 1.1239 - val_accuracy: 0.7947
Learning rate:  0.0001
Epoch 100/200
70/70 [==============================] - 294s 4s/step - loss: 0.3053 - accuracy: 0.9897 - val_loss: 1.1150 - val_accuracy: 0.7929
Learning rate:  0.0001
Epoch 101/200
70/70 [==============================] - 296s 4s/step - loss: 0.3134 - accuracy: 0.9834 - val_loss: 1.1658 - val_accuracy: 0.7841
Learning rate:  0.0001
Epoch 102/200
70/70 [==============================] - 296s 4s/step - loss: 0.3034 - accuracy: 0.9897 - val_loss: 1.1904 - val_accuracy: 0.7876
Learning rate:  0.0001
Epoch 103/200
70/70 [==============================] - 304s 4s/step - loss: 0.3149 - accuracy: 0.9838 - val_loss: 1.1910 - val_accuracy: 0.7788
Learning rate:  0.0001
Epoch 104/200
70/70 [==============================] - 297s 4s/step - loss: 0.3019 - accuracy: 0.9892 - val_loss: 1.1508 - val_accuracy: 0.7965

70/70 [==============================] - 283s 4s/step - loss: 0.2648 - accuracy: 0.9964 - val_loss: 1.1863 - val_accuracy: 0.8053
Learning rate:  1e-05
Epoch 149/200
70/70 [==============================] - 283s 4s/step - loss: 0.2623 - accuracy: 0.9964 - val_loss: 1.1831 - val_accuracy: 0.8035
Learning rate:  1e-05
Epoch 150/200
70/70 [==============================] - 283s 4s/step - loss: 0.2627 - accuracy: 0.9969 - val_loss: 1.1832 - val_accuracy: 0.8071
Learning rate:  1e-05
Epoch 151/200
70/70 [==============================] - 283s 4s/step - loss: 0.2624 - accuracy: 0.9973 - val_loss: 1.1804 - val_accuracy: 0.8071
Learning rate:  1e-05
Epoch 152/200
70/70 [==============================] - 283s 4s/step - loss: 0.2623 - accuracy: 0.9969 - val_loss: 1.1811 - val_accuracy: 0.8053
Learning rate:  1e-05
Epoch 153/200
70/70 [==============================] - 283s 4s/step - loss: 0.2687 - accuracy: 0.9937 - val_loss: 1.1812 - val_accuracy: 0.8071
Learning rate:  1e-05
Epoch 154/200
70/7

70/70 [==============================] - 283s 4s/step - loss: 0.2608 - accuracy: 0.9955 - val_loss: 1.2001 - val_accuracy: 0.8053
Learning rate:  5e-07
Epoch 198/200
70/70 [==============================] - 283s 4s/step - loss: 0.2634 - accuracy: 0.9951 - val_loss: 1.1988 - val_accuracy: 0.8053
Learning rate:  5e-07
Epoch 199/200
70/70 [==============================] - 283s 4s/step - loss: 0.2669 - accuracy: 0.9951 - val_loss: 1.1995 - val_accuracy: 0.8071
Learning rate:  5e-07
Epoch 200/200
18/18 [==============================] - 8s 438ms/step - loss: 1.1998 - accuracy: 0.8071
Test loss: 1.1997747421264648
Test accuracy: 0.8070796728134155


In [24]:
model.save('my_model_01.h5')

In [27]:
test_path = 'image_data/test'
os.listdir(path=test_path)

['0028624c49b3e0610ff9f1d111f5d532.jpg',
 '002c30700185b7971369258b438070d5.jpg',
 '00852f4f666acecd0c0d140365b42efd.jpg',
 '00c08828fce04e360c732cac01edad9e.jpg',
 '00d366e7877b6a78b104b57d67b60e6b.jpg',
 '00e803f7bc6d21b6d6d3a98136ea4635.jpg',
 '00e9cb1dca407810856e77b31309d5ab.jpg',
 '014d33090eb706769ff782d8c500dc2a.jpg',
 '015c8f0e6b95baf9dcbb34647624c5b8.jpg',
 '0194948a29f0e891c54f88004fb4c51c.jpg',
 '01964126d7cc3122173ce68761cc23bd.jpg',
 '0279619774b01b44b05b33bff44b541f.jpg',
 '027f28c9c4e255b22a8e0026cd5868b3.jpg',
 '0290c31cfc41f2dc51dcaff0dbda2da5.jpg',
 '02b5b88e51b7abd559bfb95138f33b95.jpg',
 '02b703e9b535936aa0e00886fc4669c3.jpg',
 '033925568a8d3170e7d7710483e3fae6.jpg',
 '034d01c095f88f0bcde09c3bb96682cd.jpg',
 '0370dc76bacae16e2e447b6a7549f3df.jpg',
 '03763fddc1b3b7e5751cc65398f28bf8.jpg',
 '03d2eb71b65a830092a3b6779aedbb4c.jpg',
 '03dffb85cc0a231b84e6754909e37da0.jpg',
 '041bdb3a90ae06361c3cbc246a5c291d.jpg',
 '041ce10ffd2ef73afe7e6d5fae045c98.jpg',
 '04335847221321

In [50]:
test_img_data = []
for filename in os.listdir(path=test_path):
    img = cv2.imread(test_path+'/'+filename)
    img = cv2.resize(img, (100, 100))
    test_img_data.append(img)

In [51]:
y = list_to_img(test_img_data)
y.shape

(2000, 100, 100, 3)

In [55]:
pred = model.predict(y/255)
pred = np.argmax(pred, axis=1)
pred

array([2, 4, 4, ..., 0, 3, 4], dtype=int64)

In [59]:
import csv
with open('model_01_pred.csv','w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id', 'flower_class'])
    for i, p in zip(os.listdir(path=test_path), pred):
        writer.writerow([i[:-4], p])